In [ ]:
import pyarrow as pa
import pandas as pd

df_trans = pd.read_parquet("transaction_fraud_data.parquet")
df_hist = pd.read_parquet("transaction_fraud_data.parquet")

df_trans.head()

,transaction_id,customer_id,card_number,timestamp,vendor_category,vendor_type,vendor,amount,currency,country,...,is_card_present,device,channel,device_fingerprint,ip_address,is_outside_home_country,is_high_risk_vendor,is_weekend,last_hour_activity,is_fraud
0,TX_a0ad2a2a,CUST_72886,6646734767813109,2024-09-30 00:00:01.034820,Restaurant,fast_food,Taco Bell,294.87,GBP,UK,...,False,iOS App,mobile,e8e6160445c935fd0001501e4cbac8bc,197.153.60.199,False,False,False,"{'num_transactions': 1197, 'total_amount': 334...",False
1,TX_3599c101,CUST_70474,376800864692727,2024-09-30 00:00:01.764464,Entertainment,gaming,Steam,3368.97,BRL,Brazil,...,False,Edge,web,a73043a57091e775af37f252b3a32af9,208.123.221.203,True,True,False,"{'num_transactions': 509, 'total_amount': 2011...",True
2,TX_a9461c6d,CUST_10715,5251909460951913,2024-09-30 00:00:02.273762,Grocery,physical,Whole Foods,102582.38,JPY,Japan,...,False,Firefox,web,218864e94ceaa41577d216b149722261,10.194.159.204,False,False,False,"{'num_transactions': 332, 'total_amount': 3916...",False
3,TX_7be21fc4,CUST_16193,376079286931183,2024-09-30 00:00:02.297466,Gas,major,Exxon,630.60,AUD,Australia,...,False,iOS App,mobile,70423fa3a1e74d01203cf93b51b9631d,17.230.177.225,False,False,False,"{'num_transactions': 764, 'total_amount': 2201...",False
4,TX_150f490b,CUST_87572,6172948052178810,2024-09-30 00:00:02.544063,Healthcare,medical,Medical Center,724949.27,NGN,Nigeria,...,False,Chrome,web,9880776c7b6038f2af86bd4e18a1b1a4,136.241.219.151,True,False,False,"{'num_transactions': 218, 'total_amount': 4827...",True


In [4]:
df_trans.columns

Index(['transaction_id', 'customer_id', 'card_number', 'timestamp',
       'vendor_category', 'vendor_type', 'vendor', 'amount', 'currency',
       'country', 'city', 'city_size', 'card_type', 'is_card_present',
       'device', 'channel', 'device_fingerprint', 'ip_address',
       'is_outside_home_country', 'is_high_risk_vendor', 'is_weekend',
       'last_hour_activity', 'is_fraud'],
      dtype='object')

In [ ]:
print("=== БАЗОВАЯ ИНФОРМАЦИЯ О ТРАНЗАКЦИЯХ ===")
print(f"Размер датасета: {df_trans.shape}")
print(f"Период данных: {df_trans['timestamp'].min()} - {df_trans['timestamp'].max()}")
print()

print("=== СТРУКТУРА ДАННЫХ ===")
df_trans.info()
print()

print("=== ПЕРВЫЕ 3 СТРОКИ ===")
print(df_trans.head(3))
print()

print("=== РАСПРЕДЕЛЕНИЕ МОШЕННИЧЕСТВА ===")
fraud_stats = df_trans['is_fraud'].value_counts()
fraud_rate = df_trans['is_fraud'].mean()
print(f"Fraud rate: {fraud_rate:.4f} ({fraud_rate*100:.2f}%)")
print(fraud_stats)

=== БАЗОВАЯ ИНФОРМАЦИЯ О ТРАНЗАКЦИЯХ ===
Размер датасета: (7483766, 23)
Период данных: 2024-09-30 00:00:01.034820 - 2024-10-30 23:59:59.101885

=== СТРУКТУРА ДАННЫХ ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7483766 entries, 0 to 7483765
Data columns (total 23 columns):
 #   Column                   Dtype         
---  ------                   -----         
 0   transaction_id           object        
 1   customer_id              object        
 2   card_number              int64         
 3   timestamp                datetime64[us]
 4   vendor_category          object        
 5   vendor_type              object        
 6   vendor                   object        
 7   amount                   float64       
 8   currency                 object        
 9   country                  object        
 10  city                     object        
 11  city_size                object        
 12  card_type                object        
 13  is_card_present          bool         

In [ ]:
print("=== ПРОПУЩЕННЫЕ ЗНАЧЕНИЯ ===")
missing_values = df_trans.isnull().sum()
missing_percent = (missing_values / len(df_trans)) * 100
missing_df = pd.DataFrame({
    'Missing Count': missing_values,
    'Missing Percent': missing_percent
}).sort_values('Missing Count', ascending=False)
print(missing_df[missing_df['Missing Count'] > 0])
print()

print("=== СТРУКТУРА LAST_HOUR_ACTIVITY ===")
print("Пример записи:")
import ast
sample_activity = df_trans['last_hour_activity'].iloc[0]
print(sample_activity)
print(f"Тип: {type(sample_activity)}")
print()

print("Парсинг last_hour_activity...")
try:
    activity_data = df_trans['last_hour_activity'].apply(ast.literal_eval)
    
    activity_df = pd.json_normalize(activity_data)
    print("Структура activity_df:")
    print(activity_df.info())
    print()
    print("Первые 5 строк activity_df:")
    print(activity_df.head())
    
except Exception as e:
    print(f"Ошибка при парсинге: {e}")
    print("Пробуем альтернативный способ...")
    activity_df = pd.json_normalize(df_trans['last_hour_activity'])
    print(activity_df.head())

print()

print("=== УНИКАЛЬНЫЕ ЗНАЧЕНИЯ КЛЮЧЕВЫХ ПОЛЕЙ ===")
categorical_fields = ['vendor_category', 'currency', 'country', 'city_size', 
                     'card_type', 'device', 'channel']

for field in categorical_fields:
    unique_count = df_trans[field].nunique()
    print(f"{field}: {unique_count} уникальных значений")
    if unique_count <= 20:
        print(f"  Значения: {df_trans[field].unique()}")
    else:
        print(f"  Топ-5: {df_trans[field].value_counts().head().index.tolist()}")
    print()

=== ПРОПУЩЕННЫЕ ЗНАЧЕНИЯ ===
Empty DataFrame
Columns: [Missing Count, Missing Percent]
Index: []

=== СТРУКТУРА LAST_HOUR_ACTIVITY ===
Пример записи:
{'num_transactions': 1197, 'total_amount': 33498556.080464985, 'unique_merchants': 105, 'unique_countries': 12, 'max_single_amount': 1925480.6324148502}
Тип: <class 'dict'>

Парсинг last_hour_activity...
Ошибка при парсинге: malformed node or string: {'num_transactions': 1197, 'total_amount': 33498556.080464985, 'unique_merchants': 105, 'unique_countries': 12, 'max_single_amount': 1925480.6324148502}
Пробуем альтернативный способ...
   num_transactions  total_amount  unique_merchants  unique_countries  \
0              1197  3.349856e+07               105                12   
1               509  2.011476e+07               100                12   
2               332  3.916385e+07                97                12   
3               764  2.201260e+07               105                12   
4               218  4.827636e+06               

In [ ]:
activity_df = pd.json_normalize(df_trans['last_hour_activity'])
activity_df.columns = [f'last_hour_{col}' for col in activity_df.columns]
df_trans = pd.concat([df_trans.drop('last_hour_activity', axis=1), activity_df], axis=1)

print("=== FRAUD RATE ПО КАТЕГОРИЯМ ===")

print("1. Vendor Category:")
fraud_by_category = df_trans.groupby('vendor_category').agg({
    'is_fraud': ['count', 'sum', 'mean'],
    'amount': 'mean'
}).round(4)
fraud_by_category.columns = ['total_transactions', 'fraud_count', 'fraud_rate', 'avg_amount']
fraud_by_category = fraud_by_category.sort_values('fraud_rate', ascending=False)
print(fraud_by_category)
print()

print("2. Device:")
fraud_by_device = df_trans.groupby('device').agg({
    'is_fraud': ['count', 'sum', 'mean']
}).round(4)
fraud_by_device.columns = ['total_transactions', 'fraud_count', 'fraud_rate']
fraud_by_device = fraud_by_device.sort_values('fraud_rate', ascending=False)
print(fraud_by_device)
print()

print("3. Channel:")
fraud_by_channel = df_trans.groupby('channel').agg({
    'is_fraud': ['count', 'sum', 'mean']
}).round(4)
fraud_by_channel.columns = ['total_transactions', 'fraud_count', 'fraud_rate']
print(fraud_by_channel)
print()

print("4. Булевы переменные:")
boolean_cols = ['is_card_present', 'is_outside_home_country', 'is_high_risk_vendor', 'is_weekend']
for col in boolean_cols:
    fraud_rate = df_trans.groupby(col)['is_fraud'].mean()
    print(f"{col}:")
    print(fraud_rate)
    print()

print("5. Топ стран по fraud rate:")
fraud_by_country = df_trans.groupby('country').agg({
    'is_fraud': ['count', 'sum', 'mean']
}).round(4)
fraud_by_country.columns = ['total_transactions', 'fraud_count', 'fraud_rate']
fraud_by_country = fraud_by_country.sort_values('fraud_rate', ascending=False)
print(fraud_by_country)

=== FRAUD RATE ПО КАТЕГОРИЯМ ===
1. Vendor Category:
                 total_transactions  fraud_count  fraud_rate  avg_amount
vendor_category                                                         
Travel                       935790       187477      0.2003  97273.5546
Grocery                      934029       186987      0.2002  35440.2324
Restaurant                   936178       186951      0.1997  26093.7621
Gas                          935401       186829      0.1997  45321.0558
Entertainment                936173       186890      0.1996  28830.8385
Education                    933542       186203      0.1995  45059.3506
Healthcare                   936770       186769      0.1994  44884.3147
Retail                       935883       186613      0.1994  60481.7109

2. Device:
                 total_transactions  fraud_count  fraud_rate
device                                                      
Chip Reader                  217324       217324      1.0000
NFC Payment           

In [ ]:
df_trans['hour'] = df_trans['timestamp'].dt.hour
df_trans['day_of_week'] = df_trans['timestamp'].dt.dayofweek  # 0=Monday
df_trans['date'] = df_trans['timestamp'].dt.date

print("=== ВРЕМЕННЫЕ ПАТТЕРНЫ ===")

print("1. Fraud rate по часам:")
fraud_by_hour = df_trans.groupby('hour').agg({
    'is_fraud': ['count', 'sum', 'mean']
}).round(4)
fraud_by_hour.columns = ['total_transactions', 'fraud_count', 'fraud_rate']
print(fraud_by_hour)
print()

print("2. Fraud rate по дням недели (0=Понедельник):")
fraud_by_dow = df_trans.groupby('day_of_week').agg({
    'is_fraud': ['count', 'sum', 'mean']
}).round(4)
fraud_by_dow.columns = ['total_transactions', 'fraud_count', 'fraud_rate']
print(fraud_by_dow)
print()

print("=== АНАЛИЗ СУММ ===")

print("3. Статистика сумм по типу транзакции:")
amount_stats = df_trans.groupby('is_fraud')['amount'].describe()
print(amount_stats)
print()

print("4. Медианные суммы по валютам и типу транзакции:")
amount_by_currency = df_trans.groupby(['currency', 'is_fraud'])['amount'].agg(['count', 'median', 'mean']).round(2)
print(amount_by_currency)
print()

print("5. Статистика last_hour активности:")
activity_stats = df_trans.groupby('is_fraud')[
    ['last_hour_num_transactions', 'last_hour_total_amount', 
     'last_hour_unique_merchants', 'last_hour_unique_countries',
     'last_hour_max_single_amount']
].describe()
print(activity_stats)
print()

print("6. Топ-10 дней по количеству мошенничества:")
daily_fraud = df_trans.groupby('date')['is_fraud'].sum().sort_values(ascending=False)
print(daily_fraud.head(10))

=== ВРЕМЕННЫЕ ПАТТЕРНЫ ===
1. Fraud rate по часам:
      total_transactions  fraud_count  fraud_rate
hour                                             
0                 155759        41519      0.2666
1                 280136       165999      0.5926
2                 280472       166025      0.5919
3                 280031       165621      0.5914
4                 281466       166418      0.5913
5                 208529        41829      0.2006
6                 196875        41622      0.2114
7                 301176        41318      0.1372
8                 405312        41523      0.1024
9                 352451        41522      0.1178
10                300516        41410      0.1378
11                352499        41857      0.1187
12                404942        41716      0.1030
13                352556        41829      0.1186
14                300152        41665      0.1388
15                300554        41705      0.1388
16                352379        41407      0.1175

In [ ]:
print("=== АНАЛИЗ ВАЛЮТНЫХ ДАННЫХ ===")
print("Структура курсов валют:")
print(df_hist.info())
print("\nПервые 5 строк курсов:")
print(df_hist.head())
print(f"\nПериод курсов: {df_hist['date'].min()} - {df_hist['date'].max()}")
print()

print("=== КОНВЕРТАЦИЯ В USD ===")

df_trans['date_for_join'] = df_trans['timestamp'].dt.date

df_with_rates = df_trans.merge(
    df_hist, 
    left_on='date_for_join', 
    right_on='date', 
    how='left'
)

def convert_to_usd(row):
    currency = row['currency']
    amount = row['amount']
    
    if currency == 'USD':
        return amount
    elif currency in df_hist.columns:
        rate = row[currency]
        if pd.isna(rate):
            return amount  # Если нет курса, оставляем как есть
        return amount / rate
    else:
        return amount

# Применяем конвертацию
print("Конвертируем суммы в USD...")
df_with_rates['amount_usd'] = df_with_rates.apply(convert_to_usd, axis=1)

print("Статистика сумм в USD по типу транзакции:")
usd_stats = df_with_rates.groupby('is_fraud')['amount_usd'].describe()
print(usd_stats)
print()

print("=== АНАЛИЗ СУММ В USD ===")
print("Квантили сумм в USD:")
usd_quantiles = df_with_rates.groupby('is_fraud')['amount_usd'].quantile([0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99])
print(usd_quantiles)
print()

df_with_rates['amount_usd_category'] = pd.cut(
    df_with_rates['amount_usd'], 
    bins=[0, 10, 50, 200, 1000, 5000, float('inf')],
    labels=['micro', 'very_low', 'low', 'medium', 'high', 'very_high']
)

print("Fraud rate по категориям сумм (USD):")
fraud_by_amount_cat = df_with_rates.groupby('amount_usd_category')['is_fraud'].agg(['count', 'sum', 'mean'])
fraud_by_amount_cat.columns = ['total', 'fraud_count', 'fraud_rate']
print(fraud_by_amount_cat)
print()

print("=== КОРРЕЛЯЦИИ С FRAUD ===")
numeric_cols = [
    'amount_usd', 'last_hour_num_transactions', 'last_hour_total_amount',
    'last_hour_unique_merchants', 'last_hour_unique_countries', 
    'last_hour_max_single_amount', 'hour', 'day_of_week'
]

correlations = df_with_rates[numeric_cols + ['is_fraud']].corr()['is_fraud'].sort_values(key=abs, ascending=False)
print("Корреляции с is_fraud:")
print(correlations)

=== АНАЛИЗ ВАЛЮТНЫХ ДАННЫХ ===
Структура курсов валют:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    31 non-null     object 
 1   AUD     31 non-null     float64
 2   BRL     31 non-null     float64
 3   CAD     31 non-null     float64
 4   EUR     31 non-null     float64
 5   GBP     31 non-null     float64
 6   JPY     31 non-null     float64
 7   MXN     31 non-null     float64
 8   NGN     31 non-null     float64
 9   RUB     31 non-null     float64
 10  SGD     31 non-null     float64
 11  USD     31 non-null     int64  
dtypes: float64(10), int64(1), object(1)
memory usage: 3.0+ KB
None

Первые 5 строк курсов:
         date       AUD       BRL       CAD       EUR       GBP         JPY  \
0  2024-09-30  1.443654  5.434649  1.351196  0.895591  0.747153  142.573268   
1  2024-10-01  1.442917  5.444170  1.352168  0.897557  0.746956  143.83142

C:\Users\Вера\AppData\Local\Temp\ipykernel_24164\3630750259.py:63: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  fraud_by_amount_cat = df_with_rates.groupby('amount_usd_category')['is_fraud'].agg(['count', 'sum', 'mean'])


                       total  fraud_count  fraud_rate
amount_usd_category                                  
micro                 452527       450045    0.994515
very_low              204679        27058    0.132197
low                  1505610       122232    0.081184
medium               4252642       412606    0.097023
high                 1041662       456132    0.437889
very_high              26646        26646    1.000000

=== КОРРЕЛЯЦИИ С FRAUD ===
Корреляции с is_fraud:
is_fraud                       1.000000
hour                          -0.279624
amount_usd                     0.227675
last_hour_max_single_amount    0.009226
last_hour_unique_countries     0.009046
last_hour_unique_merchants     0.006932
last_hour_num_transactions     0.004506
last_hour_total_amount         0.003332
day_of_week                   -0.000008
Name: is_fraud, dtype: float64


In [ ]:
print("=== КЛЮЧЕВЫЕ FINDINGS ===")

print("1. ВРЕМЕННЫЕ ПАТТЕРНЫ:")
night_hours = [1, 2, 3, 4]  
day_hours = [7, 8, 9, 12, 17, 18, 19]  

night_fraud_rate = df_trans[df_trans['hour'].isin(night_hours)]['is_fraud'].mean()
day_fraud_rate = df_trans[df_trans['hour'].isin(day_hours)]['is_fraud'].mean()

print(f"Ночные часы (1-4): {night_fraud_rate:.1%} fraud rate")
print(f"Дневные часы (7-9,12,17-19): {day_fraud_rate:.1%} fraud rate")
print(f"Разница: {night_fraud_rate/day_fraud_rate:.1f}x больше мошенничества ночью")
print()

print("2. ГЕОГРАФИЧЕСКИЕ ПАТТЕРНЫ:")
high_risk_countries = ['Mexico', 'Russia', 'Brazil', 'Nigeria']
low_risk_countries = ['Singapore', 'France', 'UK', 'Canada', 'Germany', 'Japan', 'USA', 'Australia']

high_risk_rate = df_trans[df_trans['country'].isin(high_risk_countries)]['is_fraud'].mean()
low_risk_rate = df_trans[df_trans['country'].isin(low_risk_countries)]['is_fraud'].mean()

print(f"Высокорисковые страны: {high_risk_rate:.1%}")
print(f"Низкорисковые страны: {low_risk_rate:.1%}")
print(f"Разница: {high_risk_rate/low_risk_rate:.1f}x больше мошенничества")
print()

print("3. СУММЫ В USD:")
print("Микротранзакции (<$10): 99.5% fraud rate - подозрительно!")
print("Очень высокие суммы (>$5000): 100% fraud rate")
print("Медиана fraud: $443 vs legitimate: $355")
print()

print("4. УСТРОЙСТВА И КАНАЛЫ:")
physical_fraud_rate = df_trans[df_trans['is_card_present'] == True]['is_fraud'].mean()
digital_fraud_rate = df_trans[df_trans['is_card_present'] == False]['is_fraud'].mean()

print(f"Физические терминалы: {physical_fraud_rate:.1%} fraud rate")
print(f"Цифровые каналы: {digital_fraud_rate:.1%} fraud rate")
print()

outside_home_fraud = df_trans[df_trans['is_outside_home_country'] == True]['is_fraud'].mean()
home_fraud = df_trans[df_trans['is_outside_home_country'] == False]['is_fraud'].mean()

print("5. ОПЕРАЦИИ ВНЕ ДОМАШНЕЙ СТРАНЫ:")
print(f"Вне дома: {outside_home_fraud:.1%}")
print(f"Дома: {home_fraud:.1%}")
print(f"Разница: {outside_home_fraud/home_fraud:.1f}x больше риска")
print()

print("=== СОЗДАНИЕ СЕГМЕНТОВ РИСКА ===")

def calculate_risk_score(row):
    score = 0

    if row['hour'] in [1, 2, 3, 4]:
        score += 3
    elif row['hour'] in [0, 5, 6, 21, 22, 23]:
        score += 1

    if row['country'] in ['Mexico', 'Russia', 'Brazil', 'Nigeria']:
        score += 2

    if row['is_outside_home_country']:
        score += 2
        
    if row['amount_usd'] < 10 or row['amount_usd'] > 5000:
        score += 2
    elif row['amount_usd'] > 2000:
        score += 1

    if not row['is_card_present']:
        score += 1
    else:
        score += 3 
        
    return min(score, 10) 

sample_df = df_with_rates.sample(100000, random_state=42)
sample_df['risk_score'] = sample_df.apply(calculate_risk_score, axis=1)

print("Fraud rate по сегментам риска (на sample):")
risk_analysis = sample_df.groupby('risk_score')['is_fraud'].agg(['count', 'sum', 'mean']).round(3)
risk_analysis.columns = ['count', 'fraud_count', 'fraud_rate']
print(risk_analysis)
print()

print("=== КОМБИНИРОВАННЫЕ РИСКИ ===")

high_risk_combo = df_with_rates[
    (df_with_rates['hour'].isin([1, 2, 3, 4])) &
    (df_with_rates['is_outside_home_country'] == True) &
    (df_with_rates['amount_usd'] > 1000)
]

if len(high_risk_combo) > 0:
    print(f"Ночь + вне дома + >$1000: {len(high_risk_combo)} транзакций")
    print(f"Fraud rate: {high_risk_combo['is_fraud'].mean():.1%}")
else:
    print("Комбинация ночь + вне дома + >$1000: недостаточно данных")

low_risk_combo = df_with_rates[
    (df_with_rates['hour'].between(9, 17)) &
    (df_with_rates['is_outside_home_country'] == False) &
    (df_with_rates['amount_usd'].between(50, 500)) &
    (df_with_rates['country'].isin(['USA', 'UK', 'Germany', 'France']))
]

print(f"День + дома + $50-500 + развитые страны: {len(low_risk_combo)} транзакций")
print(f"Fraud rate: {low_risk_combo['is_fraud'].mean():.1%}")

=== КЛЮЧЕВЫЕ FINDINGS ===
1. ВРЕМЕННЫЕ ПАТТЕРНЫ:
Ночные часы (1-4): 59.2% fraud rate
Дневные часы (7-9,12,17-19): 10.6% fraud rate
Разница: 5.6x больше мошенничества ночью

2. ГЕОГРАФИЧЕСКИЕ ПАТТЕРНЫ:
Высокорисковые страны: 37.0%
Низкорисковые страны: 7.0%
Разница: 5.2x больше мошенничества

3. СУММЫ В USD:
Микротранзакции (<$10): 99.5% fraud rate - подозрительно!
Очень высокие суммы (>$5000): 100% fraud rate
Медиана fraud: $443 vs legitimate: $355

4. УСТРОЙСТВА И КАНАЛЫ:
Физические терминалы: 100.0% fraud rate
Цифровые каналы: 12.3% fraud rate

5. ОПЕРАЦИИ ВНЕ ДОМАШНЕЙ СТРАНЫ:
Вне дома: 56.8%
Дома: 2.5%
Разница: 22.8x больше риска

=== СОЗДАНИЕ СЕГМЕНТОВ РИСКА ===
Fraud rate по сегментам риска (на sample):
            count  fraud_count  fraud_rate
risk_score                                
1           31645           48       0.002
2            8592           34       0.004
3           23875          614       0.026
4            9930          441       0.044
5            6365       